## 라이브러리 설치
### GPT-SoVITS 설치 및 드라이브 마운트

In [ ]:
# 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. GPT-SoVITS 클론 및 설치
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS
!pip install -r requirements.txt
!python install.py # 필수 모델 다운로드

In [ ]:
import json
import os
import shutil
import zipfile

# 1. 아까 만든 clean_voice_db.json 경로
JSON_PATH = "clean_voice_db.json"
# 2. 압축 파일을 저장할 경로
OUTPUT_ZIP = "training_dataset.zip"

def pack_dataset():
    print("📦 학습용 데이터 패킹 시작...")

    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        db = json.load(f)

    # 임시 폴더 생성
    temp_dir = "temp_dataset"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    count = 0
    total_files = sum(len(emotions) for emotions in db.values()) # 대략적인 수치

    # DB 구조: { "S0001": { "joy": { ... }, "sadness": { ... } } }
    for speaker_id, emotion_data in db.items():
        # 화자별 폴더 생성
        speaker_dir = os.path.join(temp_dir, speaker_id)
        os.makedirs(speaker_dir, exist_ok=True)

        for emotion, info in emotion_data.items():
            src_path = info['ref_audio_path'] # C:\dev\...

            # 파일명: 감정_원본파일.wav (파일명 충돌 방지 및 감정 식별 용이)
            original_name = os.path.basename(src_path)
            new_filename = f"{emotion}_{original_name}"
            dst_path = os.path.join(speaker_dir, new_filename)

            try:
                shutil.copy2(src_path, dst_path)
                count += 1
                if count % 100 == 0:
                    print(f"   -> 복사 중... ({count}개 완료)")
            except FileNotFoundError:
                print(f"⚠️ 파일 없음(스킵): {src_path}")

    print(f"🗜️ 압축 중... (총 {count}개 파일)")

    # zip 파일 생성
    with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(temp_dir):
            for file in files:
                file_path = os.path.join(root, file)
                # zip 안에서의 경로 (temp_dataset 제외)
                arcname = os.path.relpath(file_path, start=temp_dir)
                zipf.write(file_path, arcname)

    # 임시 폴더 삭제
    shutil.rmtree(temp_dir)
    print(f"✅ 완료! '{OUTPUT_ZIP}' 파일이 생성되었습니다.")
    print("👉 이 파일을 구글 드라이브에 업로드하세요.")

if __name__ == "__main__":
    pack_dataset()